In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

sns.set_style('darkgrid')
sns.set(font_scale=1.3)

from pathlib import Path

In [2]:
### Merge files

# Add a new column to the data containing the month of the initial file
# month_list = ['Dec-19', 'Jan-20', 'Feb-20', 'Mar-20', 'Apr-20', 'May-20', 'Jun-20', 'Jul-20', 'Aug-20', 'Sep-20', 
# 'Oct-20', 'Nov-20', 'Dec-20', 'Jan-21', 'Feb-21', 'Mar-21', 'Apr-21', 'May-21', 'Jun-21', 'Jul-21', 
# 'Aug-21', 'Sep-21', 'Oct-21', 'Nov-21', 'Dec-21', 'Jan-22', 'Feb-22']

# our_files =Path('C:\\Users\\Susan\\njeri ds\\Omdena\\conventry chapter\\coventry-chapter-effects-of-weather-main\\src\\data\\NHS Files London')

# full_df = pd.DataFrame()
# n=0
# for file in our_files.iterdir():
#     df = pd.read_excel(file)
#     df['data_month'] = month_list[n]
#     full_df = pd.concat([full_df, df], ignore_index=True)
#     n = n+1

### Cleaning and wrangling

In [3]:
data = pd.read_csv('merged_mental_data1.csv')

In [4]:
data.head(3)

,REPORTING_PERIOD_START,REPORTING_PERIOD_END,STATUS,BREAKDOWN,PRIMARY_LEVEL,PRIMARY_LEVEL_DESCRIPTION,SECONDARY_LEVEL,SECONDARY_LEVEL_DESCRIPTION,MEASURE_ID,MEASURE_NAME,MEASURE_VALUE,data_month
0,2019-12-01,2019-12-31 00:00:00,Final,Provider,RKL,WEST LONDON NHS TRUST,NONE,NONE,MHS30c,Attended contacts with memory services team in...,995,Dec-19
1,2019-12-01,2019-12-31 00:00:00,Final,Provider,RQY,SOUTH WEST LONDON AND ST GEORGE'S MENTAL HEALT...,NONE,NONE,MHS30c,Attended contacts with memory services team in...,995,Dec-19
2,2019-12-01,2019-12-31 00:00:00,Final,Provider,RAT,NORTH EAST LONDON NHS FOUNDATION TRUST,18,NONE,ACC37,Proportion of people assigned to an adult MH c...,99,Dec-19


##### Replace * in measure value into NaN and make it float datatype

In [5]:
data['MEASURE_VALUE'] = data['MEASURE_VALUE'].replace('*', np.nan)
data['MEASURE_VALUE'] = data['MEASURE_VALUE'].astype('float')

##### Convert the two date columns to datetime

In [6]:
#create new column 'period start'.
# We have to explicitly define the format because pandas infers the format for each date,
# but might mix up because it is not obvious which is the day and month

data['period_start'] = pd.to_datetime(data['REPORTING_PERIOD_START'], format='%Y-%d-%m')
data.loc[(data['REPORTING_PERIOD_START'] == '2020-06-01') | 
         (data['REPORTING_PERIOD_START'] == '2019-12-01') | 
         (data['REPORTING_PERIOD_START'] == '2020-04-01') | 
         (data['REPORTING_PERIOD_START'] == '2019-11-01') | 
         (data['REPORTING_PERIOD_START'] == '2019-10-01'), 'period_start'] = pd.to_datetime(data['REPORTING_PERIOD_START'], 
                                                                                          format='%Y-%m-%d')

In [7]:
# create new column 'period end'
# Pandas will infer the date from each column, no need to specify 
# because it is obvious which is the year, day and month for each date

data['period_end'] = pd.to_datetime(data['REPORTING_PERIOD_END'])

In [8]:
data.head(3)

,REPORTING_PERIOD_START,REPORTING_PERIOD_END,STATUS,BREAKDOWN,PRIMARY_LEVEL,PRIMARY_LEVEL_DESCRIPTION,SECONDARY_LEVEL,SECONDARY_LEVEL_DESCRIPTION,MEASURE_ID,MEASURE_NAME,MEASURE_VALUE,data_month,period_start,period_end
0,2019-12-01,2019-12-31 00:00:00,Final,Provider,RKL,WEST LONDON NHS TRUST,NONE,NONE,MHS30c,Attended contacts with memory services team in...,995.0,Dec-19,2019-12-01,2019-12-31
1,2019-12-01,2019-12-31 00:00:00,Final,Provider,RQY,SOUTH WEST LONDON AND ST GEORGE'S MENTAL HEALT...,NONE,NONE,MHS30c,Attended contacts with memory services team in...,995.0,Dec-19,2019-12-01,2019-12-31
2,2019-12-01,2019-12-31 00:00:00,Final,Provider,RAT,NORTH EAST LONDON NHS FOUNDATION TRUST,18,NONE,ACC37,Proportion of people assigned to an adult MH c...,99.0,Dec-19,2019-12-01,2019-12-31


#### Get the difference in days between the two dates (period end - period start) to get the data time-scope for each row

In [9]:
data['period_diff'] = (data['period_end'] - data['period_start'])/np.timedelta64(1, 'D')
data['period_diff'] = data['period_diff'].astype(int)

###### Note from above 
we have rows that represent monthly data (27,28,29,30), quartely (88,89,90,91), and yearly(364)

#### Clean-up the df structure: Drop the original date columns, change the column names so no need to scroll right

In [10]:
data.drop(['REPORTING_PERIOD_START','REPORTING_PERIOD_END'], axis=1, inplace=True)

In [11]:
#Convert column names to lower case and replace hyphen with a space
data.columns = data.columns.str.lower().str.replace('_', ' ')

#### Separate the data into three groups: monthly, quartely and yearly

In [12]:
mdf = data[data['period diff'] <= 31]
qdf = data[(data['period diff'] > 85) & (data['period diff'] <= 91)]
ydf = data[data['period diff'] > 360]

## Based on Anshul's Slack communication:

***..."The specific indicator name (in column I and J) you will need to look for is:
MHS01
People in contact with services at the end of the reporting period.
If you want the split for adults/children, use:
AMH01,
People in contact with adult mental health services at the end of the reporting period.
CYP01,
People in contact with children and young people's mental health services at the end of the reporting period.
(FYI, the number of adult + CYP figures will not add up to the same as the total due to unknown/missing/other data)
You also need to filter column F for the 5 London CCGs."***


We need data:
- For the 3 indicator names (column: measure ID): MHS01 (All adults and children), AMH01(adults only), and CYP01(children only)
- from the 5 London CCGs (column: primary level description)

## Monthly data

In [13]:
ccg = mdf[mdf['primary level description'].str.contains('CCG')]
ccg.shape

(45554, 13)

In [14]:
ccg['primary level description'].value_counts().sort_index()

NHS CENTRAL LONDON (WESTMINSTER) CCG    5647
NHS NORTH CENTRAL LONDON CCG            8821
NHS NORTH EAST LONDON CCG               3986
NHS NORTH WEST LONDON CCG               4094
NHS SOUTH EAST LONDON CCG               8735
NHS SOUTH WEST LONDON CCG               8640
NHS WEST LONDON CCG                     5631
Name: primary level description, dtype: int64

In [15]:
all_df = ccg[ccg['measure id'].isin(['MHS01'])] #Both adults and children
sep_df = ccg[ccg['measure id'].isin(['AMH01','CYP01'])] #Separate adults and children

In [16]:
sep_df2 = sep_df.drop(['primary level','breakdown','secondary level','secondary level description','measure name'], axis=1)
all_df2 = all_df.drop(['primary level','breakdown','secondary level','secondary level description','measure id','measure name'], 
                      axis=1)

In [17]:
sep_df2.to_csv('mental_data_ccgs_separate_adults_children_dec2019_to_feb2022.csv', index=False)
all_df2.to_csv('mental_data_ccgs_dec2019_to_feb2022.csv', index=False)

In [18]:
all_df2

,status,primary level description,measure value,data month,period start,period end,period diff
870,Final,NHS WEST LONDON CCG,4930.0,Dec-19,2019-12-01,2019-12-31,30
985,Final,NHS CENTRAL LONDON (WESTMINSTER) CCG,4440.0,Dec-19,2019-12-01,2019-12-31,30
4887,Final,NHS WEST LONDON CCG,4995.0,Jan-20,2020-01-01,2020-01-31,30
4920,Final,NHS CENTRAL LONDON (WESTMINSTER) CCG,4790.0,Jan-20,2020-01-01,2020-01-31,30
8838,Final,NHS WEST LONDON CCG,5095.0,Feb-20,2020-02-01,2020-02-29,28
...,...,...,...,...,...,...,...
174118,Performance,NHS SOUTH WEST LONDON CCG,32815.0,Jan-22,2022-01-01,2022-01-31,30
174447,Performance,NHS SOUTH EAST LONDON CCG,44955.0,Jan-22,2022-01-01,2022-01-31,30
174776,Performance,NHS NORTH CENTRAL LONDON CCG,39850.0,Jan-22,2022-01-01,2022-01-31,30
175105,Performance,NHS NORTH EAST LONDON CCG,58165.0,Jan-22,2022-01-01,2022-01-31,30


In [19]:
sep_df2

,status,primary level description,measure id,measure value,data month,period start,period end,period diff
95,Final,NHS WEST LONDON CCG,CYP01,865.0,Dec-19,2019-12-01,2019-12-31,30
498,Final,NHS CENTRAL LONDON (WESTMINSTER) CCG,CYP01,600.0,Dec-19,2019-12-01,2019-12-31,30
1034,Final,NHS WEST LONDON CCG,AMH01,4070.0,Dec-19,2019-12-01,2019-12-31,30
1106,Final,NHS CENTRAL LONDON (WESTMINSTER) CCG,AMH01,3855.0,Dec-19,2019-12-01,2019-12-31,30
4153,Final,NHS WEST LONDON CCG,CYP01,905.0,Jan-20,2020-01-01,2020-01-31,30
...,...,...,...,...,...,...,...,...
174666,Performance,NHS NORTH CENTRAL LONDON CCG,CYP01,9900.0,Jan-22,2022-01-01,2022-01-31,30
174970,Performance,NHS NORTH EAST LONDON CCG,AMH01,41645.0,Jan-22,2022-01-01,2022-01-31,30
174995,Performance,NHS NORTH EAST LONDON CCG,CYP01,11710.0,Jan-22,2022-01-01,2022-01-31,30
175299,Performance,NHS NORTH WEST LONDON CCG,AMH01,37955.0,Jan-22,2022-01-01,2022-01-31,30
